In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import gdown
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rio

import micro
import gtsa

## Download sample data

In [3]:
url = 'https://drive.google.com/drive/u/1/folders/1E1FbgeS2YHqnZOsXpbXTIpWhWT1oDR5N'
k13_dem_folder='../data/dems/k13_coreg/'
# gdown.download_folder(url, output=k13_dem_folder, quiet=False, use_cookies=False)

url = 'https://drive.google.com/drive/u/1/folders/1NBJ87RWqVEixA7XuN4uvmtjGJ03JIU56'
m14_dem_folder='../data/dems/m14_coreg/'
# gdown.download_folder(url, output=m14_dem_folder, quiet=False, use_cookies=False)

In [4]:
!tree ../data/dems

../data/dems
├── k13
│   ├── dsyr1
│   │   ├── K13_lidar_20210604_cc.tif
│   │   ├── K13_lidar_20210913_cc.tif
│   │   ├── K13_sfm_20210604_cc.tif
│   │   ├── K13_sfm_20210819_cc.tif
│   │   └── K13_sfm_20210913_cc.tif
│   ├── wsyr1
│   │   ├── K13_lidar_20201109_cc.tif
│   │   ├── K13_lidar_20210513_cc.tif
│   │   ├── K13_sfm_20201109_cc.tif
│   │   ├── K13_sfm_20210208_cc.tif
│   │   ├── K13_sfm_20210406_cc.tif
│   │   └── K13_sfm_20210513_cc.tif
│   └── wsyr2
│       ├── K13_lidar_20211007_cc.tif
│       ├── K13_lidar_20220531_cc.tif
│       ├── K13_sfm_20211007_cc.tif
│       ├── K13_sfm_20220208_cc.tif
│       ├── K13_sfm_20220503_cc.tif
│       └── K13_sfm_20220531_cc.tif
├── k13_coreg
│   ├── dsyr1
│   │   ├── k13_20210604_lidar_coreg.nc
│   │   ├── k13_20210604_lidar_coreg.tif
│   │   ├── k13_20210605_sfm_coreg.nc
│   │   ├── k13_20210605_sfm_coreg.tif
│   │   ├── k13_20210819_sfm_coreg.nc
│   │   ├── k13_20210819_sfm_coreg.tif
│   │   ├── k13_20210913_sfm_coreg.nc
│   │   └── 

In [5]:
k13_wsyr1_dem_files = sorted(Path(k13_dem_folder).glob('wsyr1/*.tif'))
k13_wsyr1_dem_files = [i.as_posix() for i in k13_wsyr1_dem_files]

k13_dsyr1_dem_files = sorted(Path(k13_dem_folder).glob('dsyr1/*.tif'))
k13_dsyr1_dem_files = [i.as_posix() for i in k13_dsyr1_dem_files]

k13_wsyr2_dem_files = sorted(Path(k13_dem_folder).glob('wsyr2/*.tif'))
k13_wsyr2_dem_files = [i.as_posix() for i in k13_wsyr2_dem_files]

m14_wsyr1_dem_files = sorted(Path(m14_dem_folder).glob('wsyr1/*.tif'))
m14_wsyr1_dem_files = [i.as_posix() for i in m14_wsyr1_dem_files]

m14_dsyr1_dem_files = sorted(Path(m14_dem_folder).glob('dsyr1/*.tif'))
m14_dsyr1_dem_files = [i.as_posix() for i in m14_dsyr1_dem_files]

m14_wsyr2_dem_files = sorted(Path(m14_dem_folder).glob('wsyr2/*.tif'))
m14_wsyr2_dem_files = [i.as_posix() for i in m14_wsyr2_dem_files]

In [6]:
k13_wsyr1_dem_files, k13_dsyr1_dem_files, k13_wsyr2_dem_files, \
m14_wsyr1_dem_files, m14_dsyr1_dem_files, m14_wsyr2_dem_files

(['../data/dems/k13_coreg/wsyr1/k13_20201109_lidar_coreg.tif',
  '../data/dems/k13_coreg/wsyr1/k13_20201110_sfm_coreg.tif',
  '../data/dems/k13_coreg/wsyr1/k13_20210208_sfm_coreg.tif',
  '../data/dems/k13_coreg/wsyr1/k13_20210406_sfm_coreg.tif',
  '../data/dems/k13_coreg/wsyr1/k13_20210513_sfm_coreg.tif'],
 ['../data/dems/k13_coreg/dsyr1/k13_20210604_lidar_coreg.tif',
  '../data/dems/k13_coreg/dsyr1/k13_20210605_sfm_coreg.tif',
  '../data/dems/k13_coreg/dsyr1/k13_20210819_sfm_coreg.tif',
  '../data/dems/k13_coreg/dsyr1/k13_20210913_sfm_coreg.tif'],
 ['../data/dems/k13_coreg/wsyr2/k13_20211007_lidar_coreg.tif',
  '../data/dems/k13_coreg/wsyr2/k13_20211008_sfm_coreg.tif',
  '../data/dems/k13_coreg/wsyr2/k13_20220208_sfm_coreg.tif',
  '../data/dems/k13_coreg/wsyr2/k13_20220503_sfm_coreg.tif',
  '../data/dems/k13_coreg/wsyr2/k13_20220531_sfm_coreg.tif'],
 ['../data/dems/m14_coreg/wsyr1/m14_20201203_lidar_coreg.tif',
  '../data/dems/m14_coreg/wsyr1/m14_20201204_sfm_coreg.tif',
  '../data/de

## Select analysis ready data
- Assuming SfM and lidar collected on the same data are only used for validation of the SfM data
- Assuming the lidar is always "better" and will thus be used for the quantitative measurements and time series analysis

In [7]:
dsm_files = ['../data/dems/k13_coreg/dsyr1/k13_20210604_lidar_coreg.tif',
  '../data/dems/k13_coreg/dsyr1/k13_20210605_sfm_coreg.tif',
  '../data/dems/k13_coreg/dsyr1/k13_20210819_sfm_coreg.tif',
  '../data/dems/k13_coreg/dsyr1/k13_20210913_sfm_coreg.tif'
            ]

## Parse time stamps

In [8]:
strings = gtsa.io.parse_timestamps(dsm_files, date_string_pattern='_........_')
date_strings = [x[1:-1] for x in strings]

# ensure chronological sorting
date_strings, dsm_files = list(zip(*sorted(zip(date_strings, dsm_files))))
date_times = [pd.to_datetime(x, format='%Y%m%d') for x in date_strings]

In [9]:
list(zip(date_strings, dsm_files))

[('20210604', '../data/dems/k13_coreg/dsyr1/k13_20210604_lidar_coreg.tif'),
 ('20210605', '../data/dems/k13_coreg/dsyr1/k13_20210605_sfm_coreg.tif'),
 ('20210819', '../data/dems/k13_coreg/dsyr1/k13_20210819_sfm_coreg.tif'),
 ('20210913', '../data/dems/k13_coreg/dsyr1/k13_20210913_sfm_coreg.tif')]

## Create stack
- resample to reference DEM grid and save as netcdf

In [10]:
k13_mask_path = '../data/vectors/kid_mask/roadClip/KID13_roadClip.shp'
m14_mask_path = '../data/vectors/mel_mask/roadClip/MEL14_roadClip.shp'

# grab shapefile crs to write to rasters
k13_crs = gpd.read_file(k13_mask_path).crs
k13_mask_gdf = gpd.read_file(k13_mask_path)

m14_crs = gpd.read_file(m14_mask_path).crs
m14_mask_gdf = gpd.read_file(m14_mask_path)

In [11]:
gdf = k13_mask_gdf

In [12]:
reference_dem = dsm_files[0]

ds = gtsa.io.xr_stack_geotifs(
    dsm_files,
    date_times,
    reference_dem,
    resampling='cubic',
    save_to_nc=True,
    overwrite=True,
    cleanup=False,
    verbose=True,
)

Resampled 4 of 4 dems to match reference DEM spatial_ref, crs, transform, bounds, and resolution.
Reading files from ../data/dems/k13_coreg/dsyr1


In [13]:
ds.rio.crs

CRS.from_epsg(32610)

In [14]:
ds = ds.rio.clip(gdf["geometry"])

## Ensure `nodata` values are correct

In [15]:
ds_nodata = ds
nd = np.nanmax(ds_nodata.isel(time=2).values)
nd = np.nanmin(ds_nodata.isel(time=2).values)
# ds_nodata = ds_nodata.where(ds_nodata!=nd)

nd

<bound method Mapping.values of <xarray.Dataset>
Dimensions:      (y: 8083, x: 1063)
Coordinates:
  * y            (y) float64 5.095e+03 5.095e+03 ... 5.015e+03 5.015e+03
  * x            (x) float64 4.99e+03 4.99e+03 4.99e+03 ... 5.001e+03 5.001e+03
    time         datetime64[ns] 2021-08-19
    spatial_ref  int64 0
Data variables:
    band1        (y, x) float64 dask.array<chunksize=(8083, 1063), meta=np.ndarray>
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0>

In [19]:
print(f"nodata: {ds_nodata['band1'].rio.nodata}")
print(f"encoded_nodata: {ds_nodata['band1'].rio.encoded_nodata}")

nodata: nan
encoded_nodata: -9999.0


In [20]:
ds_nodata['band1'].rio.write_nodata(-9999, encoded=False,inplace=True)

print(f"nodata: {ds_nodata['band1'].rio.nodata}")
print(f"encoded_nodata: {ds_nodata['band1'].rio.encoded_nodata}")

nodata: nan
encoded_nodata: -9999.0


In [23]:
for i in ds_nodata.data_vars:
                try:
                    del ds_nodata[i].attrs["grid_mapping"]
                except:
                    pass

ds_nodata.isel(time=0).rio.to_raster('../../ll_micro/test.tif')

AttributeError: 'DataArray' object has no attribute 'data_vars'

In [ ]:
ds_nodata['band1'].isel(time=0)

In [ ]:
!rio info test.tif

## Plot first time stamp

In [ ]:
ds_nodata.isel(time=0).plot(aspect='equal',
                              size=5)

## Compute differences for stack

In [ ]:
ds_diff = ds_nodata.diff(dim='time')

In [ ]:
ds_diff.isel(time=0).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')

## Apply HPGF to stacked dataset

In [ ]:
sigma = 50
ds_diff_smooth0, size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff.isel(time=0).values, sigma = sigma, origmask=True)
ds_diff_smooth1, size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff.isel(time=1).values, sigma = sigma, origmask=True)
ds_diff_smooth2, size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff.isel(time=2).values, sigma = sigma, origmask=True)
# ds_diff_smooth3, size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff.isel(time=3).values, sigma = sigma, origmask=True)
# ds_diff_smooth4, size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff.isel(time=4).values, sigma = sigma, origmask=True)
# ds_diff_smooth5, size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff.isel(time=5).values, sigma = sigma, origmask=True)

In [ ]:
ds_diff = ds_diff - [ds_diff_smooth0, ds_diff_smooth1, ds_diff_smooth2]#,\
     # ds_diff_smooth3]#, ds_diff_smooth4, ds_diff_smooth5]

## Plot first difference map

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(13,9))

ds_extent= ds_diff.rio.bounds()
m = ax[0].imshow(ds_diff.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

ax[0].imshow(ds_diff.isel(time=0), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
ax[0].set_title('KID-13\n2021-06-04 LiDAR\n2021-06-04 SfM')
ax[0].set_facecolor('k')

ax[1].imshow(ds_diff.isel(time=1), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
ax[1].set_title('KID-13\n2021-06-04 to 2021-08-19\nSfM')
ax[1].set_facecolor('k')

ax[2].imshow(ds_diff.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
ax[2].set_title('KID-13\n2021-08-19 to 2021-09-13\nSfM')
ax[2].set_facecolor('k')

# ax[3].imshow(ds_diff.isel(time=3), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[3].set_title('KID-13\n2022-05-03 to 2022-05-31\nSfM')
# ax[3].set_facecolor('k')

# ax[4].imshow(ds_diff.isel(time=4), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[4].set_title('MEL-14\n2022-04-11 to 2022-04-28\nSfM')
# ax[4].set_facecolor('k')

# ax[5].imshow(ds_diff.isel(time=5), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[5].set_title('MEL-14\n2022-04-28 to 2022-06-01\nSfM')
# ax[5].set_facecolor('k')

## Read shapefile

In [ ]:
# shapefiles = [x.as_posix() for x in sorted(Path(vector_folder).glob('*.shp'))]
# gdf = gpd.read_file(shapefiles[0])

## Plot shapefile

In [ ]:
# fig,ax = plt.subplots(figsize=(10,10))

# ds_diff.isel(time=0).plot(ax=ax,
#                           cmap='RdBu',
#                           vmin=-0.05,
#                           vmax=0.05,
#                           )

# xlim = ax.get_xlim()
# ylim = ax.get_ylim()

# gdf.plot(ax=ax, facecolor='none', edgecolor='black')

## Clip stack for each geometry

In [ ]:
# geometry_0 = gpd.GeoDataFrame(gpd.GeoSeries(gdf.iloc[0]['geometry']), columns=['geometry'])
# geometry_1 = gpd.GeoDataFrame(gpd.GeoSeries(gdf.iloc[1]['geometry']), columns=['geometry'])

In [ ]:
# ds_diff_clip_geom_0 = ds_diff.rio.clip(geometry_0['geometry'])
# ds_diff_clip_geom_1 = ds_diff.rio.clip(geometry_1['geometry'])

## Plot first time step in clipped stack

In [ ]:
# fig,ax = plt.subplots(figsize=(10,10))

# ds_diff_clip_geom_0.isel(time=0).plot(ax=ax,
#                                       cmap='RdBu',
#                                       vmin=-0.05,
#                                       vmax=0.05,
#                                       add_colorbar=False,
#                                      )

# geometry_0.plot(ax=ax, facecolor='none', edgecolor='green')

# ds_diff_clip_geom_1.isel(time=0).plot(ax=ax,
#                                       cmap='RdBu',
#                                       vmin=-0.05,
#                                       vmax=0.05,
#                                       cbar_kwargs={'extend': 'both'},
#                                      )

# geometry_1.plot(ax=ax, facecolor='none', edgecolor='purple')

# ax.set_xlim(xlim)
# ax.set_ylim(ylim);

## Plot time series for mean difference value

In [ ]:
# fig,ax = plt.subplots(figsize=(10,5))
# ds_diff_clip_geom_0.mean(dim=['x','y']).plot.scatter(ax=ax,color='green')
# ds_diff_clip_geom_1.mean(dim=['x','y']).plot.scatter(ax=ax,color='purple')

## Testing other code
Will make a new notebook for this soon.

In [ ]:
#Functions from Friedrich
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import folium
from folium.plugins import Draw
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colormaps
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D

def select_points_on_map(gdf, 
                         output_file_name = 'points.geojson'):
    
    m = gdf.explore(tiles=None,
                    style_kwds=dict(fillOpacity=0),
                    name="polygon")
    
    folium.TileLayer(
        "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
        attr='Google',
        opacity=0.8,
        name='Google basemap',
    ).add_to(m)
    

    Draw(export=True,
         filename=output_file_name,
         position = 'topleft').add_to(m)
    
    folium.LayerControl(collapsed=True).add_to(m)
    
    minimap = folium.plugins.MiniMap(position="bottomleft",)
    m.add_child(minimap)

    
    return m

def extract_linestring_coords(linestring):
    """
    Function to extract x, y coordinates from linestring object
    Input:
    shapely.geometry.linestring.LineString
    Returns:
    [x: np.array,y: np.array]
    """
    x = []
    y = []
    for coords in linestring.coords:
        x.append(coords[0])
        y.append(coords[1])
    return [np.array(x), np.array(y)]

In [ ]:
select_points_on_map(gdf, output_file_name='test.geojson')

In [ ]:
scg_points_gdf = gpd.read_file('../data/test.geojson')
scg_points_gdf.explore()

extract_linestring_coords(scg_points_gdf.to_crs(ds_nodata.rio.crs)['geometry'][0])

In [ ]:
count_da = ds_nodata.count(axis=0).compute()
std_da = ds_nodata.std(axis=0).compute()